<a href="https://colab.research.google.com/github/minhx010/minhx010/blob/main/HW1_3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sklearn
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import randrange
 

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Load Dataset
boston50data = pd.read_excel('/content/drive/MyDrive/ML_HW/boston50.xlsx')
independentVariable = pd.read_excel('/content/drive/MyDrive/ML_HW/boston50.xlsx',usecols="A:M") # create dataframe for the features

# Split a dataset into k folds
def cross_validation_split(dataset, folds=3):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / folds)
	for i in range(folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split


In [ ]:
# # test cross validation split

# seed(1)
# folds = cross_validation_split(independentVariable.values, 10)
# print(folds)

In [ ]:
# Cross Validate (10-fold)



In [14]:
# Construct the Sb_matrix
feature_means_for_each_label = boston50data.groupby(['label']).mean() # get the mean of each feature for each label
feature_means_label0 = feature_means_for_each_label.drop([1]) # mean of each feature for label 0
feature_means_label1 = feature_means_for_each_label.drop([0]) # mean of each feature for label 1

Sb_matrix = (feature_means_label1.values.reshape(13,1)-feature_means_label0.values.reshape(13,1)).dot((feature_means_label1.values.reshape(13,1)-feature_means_label0.values.reshape(13,1)).T)
# print(Sb_matrix.shape)

# Construct the Sw_matrix
first_half_of_Sw_matrix = 0
second_half_of_Sw_matrix = 0
for i in range(len(independentVariable)):
  first_half_of_Sw_matrix += ((independentVariable.iloc[i]).values.reshape(13,1)-(feature_means_label0.T).values).dot(((independentVariable.iloc[i]).values.reshape(13,1)-(feature_means_label0.T).values).T)
  second_half_of_Sw_matrix += ((independentVariable.iloc[i]).values.reshape(13,1)-(feature_means_label1.T).values).dot(((independentVariable.iloc[i]).values.reshape(13,1)-(feature_means_label1.T).values).T)
  Sw_matrix = first_half_of_Sw_matrix + second_half_of_Sw_matrix


# Calculate the eigenvalues and eigenvector for Sb and Sw (this will be used for calculating the w matrix/vector)
eigen_values, eigen_vectors = np.linalg.eig(np.linalg.inv(Sw_matrix).dot(Sb_matrix))

index = eigen_values.argsort()[::-1] # get the index of where the biggest eigenvalue is so that we could also find the biggest eigenvector - this will help us determine the direction of w
eigen_values = eigen_values[index] # order the eigenvalues
eigen_vectors = eigen_vectors[:, index]  # order the eigenvectors - matching with its eigenvalues, so then we can call on the eigenvector [0] to get our w
eigen_vectors = eigen_vectors.real 

# Get W matrix/vector (this helps us decide how many dimension we want to project the data to)
w = eigen_vectors[:,:1] # we're going to project our data 1D-wise onto this w - so that's why I only have one vector here


# Perform the linear discriminant analysis (LDA) - get Y 
Y_estimate = ((independentVariable).values).dot(w)
Y_estimate = Y_estimate.dot([1])
Y_estimate_mean = Y_estimate.mean()

for i in range(len(Y_estimate)): 
  if Y_estimate [i] < Y_estimate_mean: 
    Y_estimate [i] = 0
  elif Y_estimate [i] >= Y_estimate_mean:
    Y_estimate [i] = 1


# Calculate error
target = pd.read_excel('/content/drive/MyDrive/ML_HW/boston50.xlsx',usecols="N")
a = target.dot([1])

Error = np.linalg.norm(a-Y_estimate)
print("Error = ",Error)


Error =  20.83266665599966
